## Step 3 - Climate Analysis and Exploration

You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.

* Create a Jupyter Notebook file called `climate_analysis.ipynb` and use it to complete your climate analysis and data exporation.

* Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.

* Use SQLAlchemy `create_engine` to connect to your sqlite database.

* Use SQLAlchemy `automap_base()` to reflect your tables into classes and save a reference to those classes called `Station` and `Measurement`.

In [10]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [11]:
connection_string = 'sqlite:///hawaii.sqlite'
engine = create_engine(connection_string)

In [12]:
Base = automap_base()

In [13]:
Base.prepare(engine, reflect=True)

In [14]:
Base.classes.keys()

['hawaii_data']

In [15]:
Hawaii = Base.classes.hawaii_data

In [16]:
session = Session(engine)

In [17]:
first_row = session.query(Hawaii).first()

In [18]:
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x104d68860>,
 'date_format': datetime.datetime(2010, 1, 1, 0, 0),
 'day': 1,
 'elevation': 3,
 'id': 1,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'month': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65,
 'year': 2010}